In [1]:
# 引入库包
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('data/train.csv',header=None)
df_train.columns = ['PassengerId','Survived','Pclass','Name','Sex','Age',"SibSp","Parch","Ticket","Fare","Cabin","Embarked"]
df_train.drop(index=0,inplace=True)
df_train.head(5)
#PassengerId:乘客编号
#Survived：存活情况（存活：1，死亡：0）
#Pclass：客舱等级
#Name：乘客姓名
#Sex：性别
#Age：年龄
#SibSp：同乘的兄弟姐妹/配偶数
#Parch：同乘的父母/小孩数
#Ticket：船票编号
#Fare：船票价格
#Cabin：客舱号
#Embarked：登船港口

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,NaN,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
3,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,NaN,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
5,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,NaN,S


In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 12 columns):
PassengerId    891 non-null object
Survived       891 non-null object
Pclass         891 non-null object
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null object
SibSp          891 non-null object
Parch          891 non-null object
Ticket         891 non-null object
Fare           891 non-null object
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: object(12)
memory usage: 90.5+ KB


In [4]:
df_train.select_dtypes(include="object").describe().T.assign(
  missing_pct=df_train.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,unique,top,freq,missing_pct
PassengerId,891,891,493,1,0.000000
Survived,891,2,0,549,0.000000
Pclass,891,3,3,491,0.000000
Name,891,891,"Becker, Miss. Marion Louise",1,0.000000
Sex,891,2,male,577,0.000000
Age,714,88,24,30,0.198653
SibSp,891,7,0,608,0.000000
Parch,891,7,0,678,0.000000
Ticket,891,681,347082,7,0.000000
Fare,891,248,8.05,43,0.000000


In [5]:
df_train['Title'] = df_train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
title_Dict = {}
title_Dict.update(dict.fromkeys(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer'))
title_Dict.update(dict.fromkeys(['Don', 'Sir', 'the Countess','Countess', 'Dona', 'Lady'], 'Royalty'))
title_Dict.update(dict.fromkeys(['Mme', 'Ms', 'Mrs'], 'Mrs'))
title_Dict.update(dict.fromkeys(['Mlle', 'Miss'], 'Miss'))
title_Dict.update(dict.fromkeys(['Mr'], 'Mr'))
title_Dict.update(dict.fromkeys(['Master','Jonkheer'], 'Master'))

df_train['Title'] = df_train['Title'].map(title_Dict)

In [6]:
df_train.select_dtypes(include="object").describe().T.assign(
  missing_pct=df_train.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,unique,top,freq,missing_pct
PassengerId,891,891,493,1,0.000000
Survived,891,2,0,549,0.000000
Pclass,891,3,3,491,0.000000
Name,891,891,"Becker, Miss. Marion Louise",1,0.000000
Sex,891,2,male,577,0.000000
Age,714,88,24,30,0.198653
SibSp,891,7,0,608,0.000000
Parch,891,7,0,678,0.000000
Ticket,891,681,347082,7,0.000000
Fare,891,248,8.05,43,0.000000


In [7]:
df_train.drop("PassengerId",1,inplace=True)
df_train.drop("Name",1,inplace=True)
df_train.drop("Ticket",1,inplace=True)

In [8]:
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
1,0,3,male,22,1,0,7.25,NaN,S,Mr
2,1,1,female,38,1,0,71.2833,C85,C,Mrs
3,1,3,female,26,0,0,7.925,NaN,S,Miss
4,1,1,female,35,1,0,53.1,C123,S,Mrs
5,0,3,male,35,0,0,8.05,NaN,S,Mr


In [9]:
df_train.select_dtypes(include="object").describe().T.assign(
  missing_pct=df_train.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,unique,top,freq,missing_pct
Survived,891,2,0,549,0.000000
Pclass,891,3,3,491,0.000000
Sex,891,2,male,577,0.000000
Age,714,88,24,30,0.198653
SibSp,891,7,0,608,0.000000
Parch,891,7,0,678,0.000000
Fare,891,248,8.05,43,0.000000
Cabin,204,147,C23 C25 C27,4,0.771044
Embarked,889,3,S,644,0.002245
Title,891,6,Mr,517,0.000000


In [10]:
df_train.drop("Cabin",1,inplace=True)

In [11]:
df_train.Embarked.isnull().value_counts()

False    889
True       2
Name: Embarked, dtype: int64

In [12]:
#删除数据集中某行的特征
def drop_feature_null_rows(df,feature):
    df[feature] = df[feature].fillna("999")#登船港口
    null_rows_index = df[(df[feature] == "999")].index.tolist()
    return df.drop(null_rows_index)

#删除登船港口的空行
df_train =drop_feature_null_rows(df_train,'Embarked')
df_train.shape

(889, 9)

In [13]:
df_train.select_dtypes(include="object").describe().T.assign(
  missing_pct=df_train.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,unique,top,freq,missing_pct
Survived,889,2,0,549,0.0000
Pclass,889,3,3,491,0.0000
Sex,889,2,male,577,0.0000
Age,712,88,24,30,0.1991
SibSp,889,7,0,606,0.0000
Parch,889,7,0,676,0.0000
Fare,889,247,8.05,43,0.0000
Embarked,889,3,S,644,0.0000
Title,889,6,Mr,517,0.0000


In [14]:
df_train['Fare'] = df_train['Fare'].astype("float")
df_train['Pclass'] = df_train['Pclass'].astype("int64")

In [15]:
from sklearn.preprocessing import LabelEncoder

# 建立PClass Fare Category
def pclass_fare_category(df, pclass1_mean_fare, pclass2_mean_fare, pclass3_mean_fare):
    if df['Pclass'] == 1:
        if df['Fare'] <= pclass1_mean_fare:
            return 'Pclass1_Low'
        else:
            return 'Pclass1_High'
    elif df['Pclass'] == 2:
        if df['Fare'] <= pclass2_mean_fare:
            return 'Pclass2_Low'
        else:
            return 'Pclass2_High'
    elif df['Pclass'] == 3:
        if df['Fare'] <= pclass3_mean_fare:
            return 'Pclass3_Low'
        else:
            return 'Pclass3_High'

Pclass1_mean_fare = df_train['Fare'].groupby(by=df_train['Pclass']).mean().get([1]).values[0]
Pclass2_mean_fare = df_train['Fare'].groupby(by=df_train['Pclass']).mean().get([2]).values[0]
Pclass3_mean_fare = df_train['Fare'].groupby(by=df_train['Pclass']).mean().get([3]).values[0]

# 建立Pclass_Fare Category
df_train['Pclass_Fare_Category'] = df_train.apply(pclass_fare_category, args=(
    Pclass1_mean_fare, Pclass2_mean_fare, Pclass3_mean_fare), axis=1)

pclass_level = LabelEncoder()

# 给每一项添加标签
pclass_level.fit(np.array(['Pclass1_Low', 'Pclass1_High', 'Pclass2_Low', 'Pclass2_High', 'Pclass3_Low', 'Pclass3_High']))

# 转换成数值
df_train['Pclass_Fare_Category'] = pclass_level.transform(df_train['Pclass_Fare_Category'])

# dummy 转换
df_train = pd.get_dummies(df_train,columns = ['Pclass_Fare_Category'])

In [16]:
#将 Pclass 特征factorize化
#df_train['Pclass'] = pd.factorize(df_train['Pclass'])[0]

In [17]:
df_train.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'Title', 'Pclass_Fare_Category_0', 'Pclass_Fare_Category_1',
       'Pclass_Fare_Category_2', 'Pclass_Fare_Category_3',
       'Pclass_Fare_Category_4', 'Pclass_Fare_Category_5'],
      dtype='object')

In [18]:
df_train = pd.get_dummies(df_train,columns = ["Sex","Pclass","Embarked","Title"])
df_train.head()

,Survived,Age,SibSp,Parch,Fare,Pclass_Fare_Category_0,Pclass_Fare_Category_1,Pclass_Fare_Category_2,Pclass_Fare_Category_3,Pclass_Fare_Category_4,...,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
1,0,22,1,0,7.2500,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
2,1,38,1,0,71.2833,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,1,26,0,0,7.9250,0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,0
4,1,35,1,0,53.1000,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0
5,0,35,0,0,8.0500,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0


In [19]:
df_train.columns

Index(['Survived', 'Age', 'SibSp', 'Parch', 'Fare', 'Pclass_Fare_Category_0',
       'Pclass_Fare_Category_1', 'Pclass_Fare_Category_2',
       'Pclass_Fare_Category_3', 'Pclass_Fare_Category_4',
       'Pclass_Fare_Category_5', 'Sex_female', 'Sex_male', 'Pclass_1',
       'Pclass_2', 'Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer',
       'Title_Royalty'],
      dtype='object')

In [20]:
df_train.fillna(0,inplace=True)

In [21]:
from sklearn import preprocessing

#归一化处理
min_max_scaler = preprocessing.MinMaxScaler()
X_temp = min_max_scaler.fit_transform(df_train[["Fare","SibSp","Parch","Age"]])
#标准化处理
df_train[["Fare","SibSp","Parch","Age"]] = preprocessing.scale(X_temp)

In [22]:
df_train.head()

,Survived,Age,SibSp,Parch,Fare,Pclass_Fare_Category_0,Pclass_Fare_Category_1,Pclass_Fare_Category_2,Pclass_Fare_Category_3,Pclass_Fare_Category_4,...,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
1,0,-0.099150,0.431350,-0.474326,-0.500240,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
2,1,0.812389,0.431350,-0.474326,0.788947,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,1,0.128735,-0.475199,-0.474326,-0.486650,0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,0
4,1,0.641476,0.431350,-0.474326,0.422861,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0
5,0,0.641476,-0.475199,-0.474326,-0.484133,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0


In [23]:
Y = df_train.Survived
X = df_train.drop('Survived',1,inplace=False)

In [24]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=123)

# LR 模型训练

In [25]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(x_train, y_train)

print(u'LR模型的平均正确率为：%s' % lr.score(x_test, y_test))

LR模型的平均正确率为：0.8089887640449438


# 预测

In [26]:
df_test = pd.read_csv('data/test.csv',header=None)
df_test.columns = ['PassengerId','Pclass','Name','Sex','Age',"SibSp","Parch","Ticket","Fare","Cabin","Embarked"]
df_test.drop(index=0,inplace=True)

In [27]:
df_temp = df_test.copy()

In [28]:
df_test.drop("PassengerId",1,inplace=True)

df_test['Title'] = df_test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df_test['Title'] = df_test['Title'].map(title_Dict)
df_test.drop("Name",1,inplace=True)

df_test.drop("Ticket",1,inplace=True)
df_test.drop("Cabin",1,inplace=True)

df_test['Fare'] = df_test['Fare'].astype("float")
df_test['Fare'] = df_test[['Fare']].fillna(df_test.groupby('Pclass').transform(np.mean))


df_test['Fare'] = df_test['Fare'].astype("float")
df_test['Pclass'] = df_test['Pclass'].astype("int64")

Pclass1_mean_fare = df_test['Fare'].groupby(by=df_test['Pclass']).mean().get([1]).values[0]
Pclass2_mean_fare = df_test['Fare'].groupby(by=df_test['Pclass']).mean().get([2]).values[0]
Pclass3_mean_fare = df_test['Fare'].groupby(by=df_test['Pclass']).mean().get([3]).values[0]

# 建立Pclass_Fare Category
df_test['Pclass_Fare_Category'] = df_test.apply(pclass_fare_category, args=(
    Pclass1_mean_fare, Pclass2_mean_fare, Pclass3_mean_fare), axis=1)
pclass_level = LabelEncoder()

# 给每一项添加标签
pclass_level.fit(np.array(['Pclass1_Low', 'Pclass1_High', 'Pclass2_Low', 'Pclass2_High', 'Pclass3_Low', 'Pclass3_High']))

# 转换成数值
df_test['Pclass_Fare_Category'] = pclass_level.transform(df_test['Pclass_Fare_Category'])

# dummy 转换
df_test = pd.get_dummies(df_test,columns = ['Pclass_Fare_Category'])

df_test = pd.get_dummies(df_test,columns = ["Sex","Pclass","Embarked","Title"])
df_test.fillna(0,inplace=True)

print(df_test.columns)

Index(['Age', 'SibSp', 'Parch', 'Fare', 'Pclass_Fare_Category_0',
       'Pclass_Fare_Category_1', 'Pclass_Fare_Category_2',
       'Pclass_Fare_Category_3', 'Pclass_Fare_Category_4',
       'Pclass_Fare_Category_5', 'Sex_female', 'Sex_male', 'Pclass_1',
       'Pclass_2', 'Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer',
       'Title_Royalty'],
      dtype='object')


In [29]:
from sklearn import preprocessing

#归一化处理
min_max_scaler = preprocessing.MinMaxScaler()
X_temp = min_max_scaler.fit_transform(df_test[["Fare","SibSp","Parch","Age"]])

#标准化处理
df_test[["Fare","SibSp","Parch","Age"]] = preprocessing.scale(X_temp)

In [30]:
Predict = lr.predict(df_test)

In [31]:
df_test.columns

Index(['Age', 'SibSp', 'Parch', 'Fare', 'Pclass_Fare_Category_0',
       'Pclass_Fare_Category_1', 'Pclass_Fare_Category_2',
       'Pclass_Fare_Category_3', 'Pclass_Fare_Category_4',
       'Pclass_Fare_Category_5', 'Sex_female', 'Sex_male', 'Pclass_1',
       'Pclass_2', 'Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer',
       'Title_Royalty'],
      dtype='object')

In [32]:
# submission
df = pd.DataFrame({"PassengerId": df_temp["PassengerId"].values, "Survived": Predict})
df.to_csv("gender_submission.csv", index=False)